In [1]:
def add_halo_properties(gal, hal):
    gal.meta.__dict__['idx'] = hals.data['idx'][i]
    gal.meta.__dict__['rhalo'] = hals.data['rvir'][i]

In [2]:
import pickle

In [5]:
prg_only_tree = pickle.load(open(wdir+'prg_only_tree.pickle', 'rb'))

In [9]:
import tree.ctutils as ctu

In [10]:
aprg = ctu.extract_main_tree(prg_only_tree, idx=43071)

In [13]:
aprg.dtype

dtype([('aexp', '<f8'), ('id', '<i8'), ('desc_aexp', '<i8'), ('desc_id', '<i8'), ('nprog', '<i8'), ('pid', '<i8'), ('upid', '<i8'), ('desc_pid', '<i8'), ('phantom', '<i8'), ('sam_mvir', '<f8'), ('mvir', '<f8'), ('rvir', '<f8'), ('rs', '<i8'), ('vrms', '<f8'), ('mmp', '<f8'), ('aexp_last_MM', '<f8'), ('vmax', '<i8'), ('x', '<f8'), ('y', '<f8'), ('z', '<f8'), ('vx', '<f8'), ('vy', '<f8'), ('vz', '<f8'), ('jx', '<f8'), ('jy', '<f8'), ('jz', '<f8'), ('spin', '<f8'), ('b_id', '<i8'), ('d_id', '<i8'), ('tree_root_id', '<i8'), ('Orig_halo_id', '<i8'), ('nout', '<i8'), ('next_coprogenitor_d_id', '<i8'), ('last_progenitor_d_id', '<i8'), ('last_mainleaf_depthfirst_id', '<i8'), ('tidal_force', '<f8'), ('tidal_id', '<i8'), ('np', '<i4'), ('r', '<f4'), ('tvir', '<f4'), ('cvel', '<f4'), ('sig', '<f4'), ('sigbulge', '<f4'), ('mbulge', '<f4'), ('m', '<f4')])

In [12]:
import load
import numpy as np
import tree.halomodule as hmo

wdir = './29172/'
nout = 180

info = load.info.Info(nout=nout, base=wdir)

gcat = hmo.Halo(base=wdir, is_gal=True, verbose=True, nout=nout)

gg = gcat.data[10]
galid = gg['id']

import galaxy

# galdata is needed to set up the meta data of the galaxy. 
# modify the code to make the intend clearer.
# 
# Finally a galaxy instance has self.star, self.dm, self.cell, self.meta and other properties and methods.
# 
gm = load.rd_GM.rd_gal(nout, galid, base=wdir)
gm.dm = load.rd_GM.rd_dm(nout, halid, base=wdir)
#gm.cell = load.rd_GM.rd_cell(nout, galid, base=wdir)

[Halo.load_info] loading info
[Halo.load_info] nout = 180, base =./29172/
[Halo.load_info] info is loaded
180
Loading file: ./29172/GalaxyMaker/gal/tree_bricks180


NameError: name 'halid' is not defined

In [20]:
print(min(gm.star['vx']), max(gm.star['vx']))
print(min(gm.cell['var1']), max(gm.cell['var1']))
gal = galaxy.galaxy.Galaxy(halo = gg, radius_method='eff', info=info)


good_gal = gal.mk_gal(gm.star, None, gm.cell, unit_conversion="GM", verbose=True)
print(min(gal.cell['vx']), max(gal.cell['vx']))

106.352600098 677.018432617
0.0084149377569 0.0325635425693
Making a galaxy: 11
SAVE: False
RSCALE: 0.5
Halo size: 43.7766474071
0.511899 0.689009 0.459023
Rgal_tmp 43.7766474071
nstar tot: 33806
Store stellar particle
Cell is NOT none
-217.864201957 282.625032292


/home/hoseung/Work/pyclusterevol/galaxy/galaxy.py:123: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  m_radial[i] = np.sum(m_sorted[ibins[i]:ibins[i+1]])


In [11]:
print(min(gal.cell['vz']), max(gal.cell['vz']))
print(min(gal.star['vz']), max(gal.star['vz']))

-1134.63759391 222.49821016
-1045.18783686 488.962340145


## Now, further calculations

In [4]:
## Parameters
lambda_params = {'npix_per_reff':5, "rscale":3, 'method':'ellip',
                'verbose':False}



# general
galaxy_plot=True
galaxy_plot_dir = './'
dump_gal=False # I don't need it anymore. (but I can save meta data of a galaxy, where meta = all - star - dm - cell.)
reorient=True


import queue
out_q = queue.Queue()

### Units

* Often data come through different pipelines have different units. 
But don't try to devise a nice method to deal with units of ndarray.
close to hundred of ndarray attributes will need wrappers.

In [6]:
import galaxy.make_gal as mkg


lambdas = gal.cal_lambda_r_eps(**lambda_params) # calculate within 1.0 * reff
gal.meta.lambda_arr, gal.meta.lambda_arrh, gal.meta.lambda_12kpc= lambdas[0]
gal.meta.lambda_r,   gal.meta.lambda_rh,   gal.meta.lambda_12kpc = lambdas[1]
if galaxy_plot:
    gal.plot_gal(fn_save = galaxy_plot_dir + str(nout).zfill(3) \
                     + "_" + str(galid) + ".png", ioff=True)

#add_halo_properties(gal, hal)
#print("\n \n Good galaxy. ID, IDx", galid, gal.meta.idx)


if dump_gal:
    # save gal before reorientation
    print(galaxy_plot_dir + str(nout).zfill(3) + "_" + str(galid) + ".h5")
    mkg.save_gal(gal, galaxy_plot_dir + str(nout).zfill(3) + "_" + str(galid) + ".h5")

self.rscale_lambda is updated to 3.
effective radius 8.03799983382 32.1519993353
100.00% of stellar particles selected
frac05 0.123125
frac15 0.303902242168
Updating data
minmax field after crop and converting into physical unit 4695214.53972 122075578.033
Updating data


In [7]:
if reorient:
    gal.cal_norm_vec(["star"], dest=[0.,1.,0.])
    gal.cal_rotation_matrix(dest = [0., 1., 0.])
    gal.reorient(dest=[0., 1., 0.], pop_nvec = ['star'],
                 pops=['star', 'cell'], verbose=False)
    lambdas = gal.cal_lambda_r_eps(**lambda_params)
# Because lambda is measured for ALL stars,
# B/T must also be measured for ALL stars, not only for bound stars.
#                print(lambdas[0],"d", lambdas[1])
    gal.meta.lambda_arr2, gal.meta.lambda_arr2h, gal.meta.lambda_arr2q = lambdas[0]
    gal.meta.lambda_r2,   gal.meta.lambda_r2h  , gal.meta.lambda_r2q   = lambdas[1]
if galaxy_plot:

#                gal.cal_b2t(ptype='star',
#                            bound_only=False,
#                            hist=False,
#                            proj='y')
    gal.plot_gal(fn_save = galaxy_plot_dir + str(nout).zfill(3) \
                         + "_" + str(galid) + "_reori.png", ioff=True)
out_q.put(gal.meta.__dict__)

self.rscale_lambda is updated to 3.
effective radius 8.03799983382 32.1519993353
100.00% of stellar particles selected
frac05 0.10125
frac15 0.303902242168
Updating data
minmax field after crop and converting into physical unit 4657205.04201 149030561.344
Updating data


In [7]:
a = out_q.get()

KeyboardInterrupt: 

###  seems good so far.
Now gas properties

In [9]:
def gas_inflow(cell):
    """
        How do I measure gas? 
        
        Spherical shell?
        isosurface? 
        
    """
    
    
    

In [4]:
from draw import pp
import importlib

In [20]:
importlib.reload(pp)

<module 'draw.pp' from '/home/hoseung/Work/pyclusterevol/draw/pp.py'>

In [7]:
dmap = pp.pp_cell(gal.cell, 82, gal.info, verbose=True)

 ... working resolution % npix  72 72
 ... given npix: 82
int32 int32
dimensions of 
 mass = (6370,), 
 sden = (6370,), and nx, ny are 72, 72
True True
[ 279.69843447  293.21163991  345.80081569  253.51006741  342.82538227
  337.03465078  338.79938971  255.40765138  246.93886291  337.52080765]
[ 0.02414744  0.02653709  0.03690991  0.01983725  0.03627746  0.03506227
  0.03543041  0.02013533  0.01882218  0.0351635 ]
int32 int32 int32 int32 int32


/home/hoseung/Work/pyclusterevol/draw/pp.py:646: RuntimeWarning: invalid value encountered in true_divide
  return resize(ppc.col_over_denom(iin, ixl, ixr, iyl, iyr, mass, sden, nx, ny, column), [npix,npix])
/home/hoseung/Work/pyclusterevol/draw/pp.py:356: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  Y[i,:] = X[i*p,np.int_(np.arange(n)*q)]


In [8]:
import matplotlib.pyplot as plt

In [9]:
plt.imshow(dmap)
plt.show()

In [ ]:
import vispy

In [ ]:
# 3D volume rendering using vispy/volume_rendering example. 
# simple (uniform grid) 3D array is needed.